<a href="https://colab.research.google.com/github/hyunjyyyy/smdata/blob/main/smdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sys

program_file = "1) 비교과 프로그램 정보.xlsx"
completer_file = "2) 비교과 이수자 정보.xlsx"

# 3. 엑셀 파일 로드
try:
    # 1) 비교과 프로그램 정보.xlsx 파일 로드
    df_programs = pd.read_excel(program_file, sheet_name='Sheet1')

    # 2) 비교과 이수자 정보.xlsx 파일 로드
    df_completers = pd.read_excel(completer_file, sheet_name='sheet1')

    print(f"--- 엑셀 파일 로드 성공 ---")
    print(f"'{program_file}' (Sheet1) 로드 완료: {df_programs.shape} (행, 열)")
    print(f"'{completer_file}' (sheet1) 로드 완료: {df_completers.shape} (행, 열)")

except FileNotFoundError as e:
    print(f"!!! [오류] 파일을 찾을 수 없습니다: {e}")
    print("!!! -> 왼쪽 '파일' 탭에 엑셀 파일 2개가 모두 정확한 이름으로 업로드되었는지 확인해주세요.")
    sys.exit() # 오류 발생 시 중단
except ValueError as e:
    print(f"!!! [오류] 엑셀 시트 이름을 찾을 수 없습니다: {e}")
    print("!!! -> 엑셀 파일의 시트 이름이 'Sheet1' 또는 'sheet1'이 맞는지 확인해주세요.")
    sys.exit() # 오류 발생 시 중단
except Exception as e:
    print(f"!!! [알 수 없는 오류] 파일 로드 중 문제가 발생했습니다: {e}")
    sys.exit() # 오류 발생 시 중단


# 4. 전처리: 프로그램 정보 (df_programs)
print("\n--- '프로그램 정보' 전처리 시작 ---")
# '시작일', '종료일'을 날짜 형식으로 변환 (오류 시 무시)
df_programs['시작일'] = pd.to_datetime(df_programs['시작일'], errors='coerce')
df_programs['종료일'] = pd.to_datetime(df_programs['종료일'], errors='coerce')

# 핵심역량점수 컬럼 리스트
core_cols = [f'핵심역량점수{i}' for i in range(1, 9)]

# df_programs의 핵심역량점수 결측치를 0으로 채우기
df_programs[core_cols] = df_programs[core_cols].fillna(0)
print("'프로그램 정보' 전처리 완료.")


# 5. 전처리: 이수자 정보 (df_completers)
print("\n--- '이수자 정보' 전처리 시작 ---")
# '핵심역량점수' 컬럼 결측치 0으로 채우기
df_completers[core_cols] = df_completers[core_cols].fillna(0)

# 이수자ID string으로 변환
df_completers['이수자ID'] = df_completers['이수자ID'].astype(str)
print("'이수자 정보' 전처리 완료.")


# 6. 데이터 병합
print("\n--- 데이터 병합 시작 ---")
# 프로그램ID와 프로그램세부ID를 기준으로 두 데이터 벙합
df_merged = pd.merge(
    df_completers,  # 이수자 정보 (Left)
    df_programs,    # 프로그램 정보 (Right)
    on=['프로그램ID', '프로그램세부ID'], # 병합 기준 키
    how='left',      # 이수자 정보(df_completers)를 기준으로
    suffixes=('_이수자', '_프로그램') # 중복 컬럼명(예: 마일리지) 구분
)
print(f"병합 완료. 최종 데이터: {df_merged.shape} (행, 열)")


# 7. 최종 결과 파일로 저장
output_filename = "preprocessed_merged_data.csv"
df_merged.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"'{output_filename}' 파일이 코랩 환경에 저장되었습니다.")

--- 엑셀 파일 로드 성공 ---
'1) 비교과 프로그램 정보.xlsx' (Sheet1) 로드 완료: (939, 26) (행, 열)
'2) 비교과 이수자 정보.xlsx' (sheet1) 로드 완료: (94469, 17) (행, 열)

--- '프로그램 정보' 전처리 시작 ---
'프로그램 정보' 전처리 완료.

--- '이수자 정보' 전처리 시작 ---
'이수자 정보' 전처리 완료.

--- 데이터 병합 시작 ---
병합 완료. 최종 데이터: (94469, 41) (행, 열)
'preprocessed_merged_data.csv' 파일이 코랩 환경에 저장되었습니다.


In [ ]:
import pandas as pd
import io # df.info()의 출력을 깔끔하게 받기 위해 import 합니다.

# 1. 파일 이름 (업로드한 파일)
filename = "preprocessed_merged_data.csv"

try:
    # 2. 파일 읽기
    df = pd.read_csv(filename)
    print(f"--- '{filename}' 파일 로드 성공 ---")
    print(f"전체 데이터: {df.shape[0]}행, {df.shape[1]}열\n")

    # 3. 데이터 전반적인 정보 (컬럼명, Dtype, 결측치)
    print("--- 1. 데이터 컬럼 및 결측치 정보 (df.info()) ---")
    # df.info()는 결과를 바로 출력(print)합니다.
    # 출력을 변수로 받아 print()로 제어하기 위해 StringIO를 사용합니다.
    buffer = io.StringIO()
    df.info(buf=buffer)
    print(buffer.getvalue())

    # 4. 숫자형 데이터 분포 (기술 통계)
    print("\n--- 2. 숫자형 데이터 기술 통계 (df.describe()) ---")
    # .T (Transpose)를 붙이면 행/열이 바뀌어 목록처럼 보기 편합니다.
    print(df.describe().T)

    # 5. 범주형(문자열) 데이터 분포 (고유값, 최빈값)
    print("\n--- 3. 범주형(문자열) 데이터 기술 통계 (df.describe(include='object')) ---")
    # 'object'는 pandas에서 문자열 타입을 의미합니다.
    print(df.describe(include=['object']).T)

except FileNotFoundError:
    print(f"!!! [오류] '{filename}' 파일을 찾을 수 없습니다.")
    print("!!! -> 코랩 왼쪽 '파일' 탭에 파일이 업로드되었는지 다시 확인해주세요.")
except Exception as e:
    print(f"데이터 분석 중 오류 발생: {e}")

--- 'preprocessed_merged_data.csv' 파일 로드 성공 ---
전체 데이터: 94469행, 41열

--- 1. 데이터 컬럼 및 결측치 정보 (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94469 entries, 0 to 94468
Data columns (total 41 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   프로그램ID        94469 non-null  int64  
 1   프로그램세부ID      94469 non-null  int64  
 2   단과대학명         94469 non-null  object 
 3   전공명           94469 non-null  object 
 4   학년            94469 non-null  int64  
 5   이수자ID         94469 non-null  object 
 6   만족도평균         38761 non-null  float64
 7   이수여부          94469 non-null  object 
 8   마일리지_이수자      94469 non-null  int64  
 9   핵심역량점수1_이수자   94469 non-null  float64
 10  핵심역량점수2_이수자   94469 non-null  float64
 11  핵심역량점수3_이수자   94469 non-null  float64
 12  핵심역량점수4_이수자   94469 non-null  float64
 13  핵심역량점수5_이수자   94469 non-null  float64
 14  핵심역량점수6_이수자   94469 non-null  float64
 15  핵심역량점수7_이수자   94469 non-null  float64
 16  핵심역량

94,469개의 활동, 41개의 컬럼

2. 숫자형 데이터 분포
- 학년: 평균 2.7년(3학년 50%, 2학년 25%)
- 만족도 평균: 평균 4.9(75%가 5점), 만족도는 38,761건만 응답 있고 나머지는 무응답
- 마일리지 이수자: 절반정도만 마일리지 수령
- 핵심역량점수: 0.1~0.9 사이

---
3. 문자열 데이터 통계

- 공과대학 참여기록 가장 많음
- 경영학부 참여기록 가장 많음
- 57506/94469 = 이수율
- '폭력 예방교육' 가장 많이 신청
- '대학일자리플러스센터'가 가장 많은 프로그램 주관
- 비대면 방식이 절반 이상

---
- content-based flitering: 이제까지 들은 비교과 기반 추천
- collaborative filtering: 나와 비슷한 학생들의 선택 추천해주기
- 내 전공/학년 맞춤 기반 추천(공대 2학년..)

